based on: https://www.youtube.com/watch?v=4QkYy1wANXA

# Sportan - Cricket Data Analytics
In this project the problem is defined as "We don't know the strengths and weaknesses of our opponents but give me the best 11 from this planet". 
1) The team should be able to score at least 180 runs on an average
2) They should be to defend 150 runs on an average


### Step 1 - Data Collection
 Data was downloaded from: 
https://github.com/techyweeb993/Power-Bi---T20-Cricket-Data-Analysis/tree/main

In [33]:
# Import necessary libraries
import pandas as pd
import json

### Step 2 - Preprocessing


Create Match DataFrame

In [34]:
# Load data and create DataFrame for MATCH RESULTS
with open('Data/t20_json_files/t20_wc_match_results.json') as f:
    data = json.load(f)

df_match = pd.DataFrame(data[0]['matchSummary'])

# Scorecard will be used for a id/key for connection between the tables
df_match.rename({'scorecard':'match_id'}, axis=1, inplace=True)

df_match.head(4)

,team1,team2,winner,margin,ground,matchDate,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,"Oct 16, 2022",T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,"Oct 16, 2022",T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,"Oct 17, 2022",T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,"Oct 17, 2022",T20I # 1828


In [35]:
# Need to create dict which allows for link two tables
match_ids_dict = {}

for index, row in df_match.iterrows():
    match_ids_dict[row["team1"] + ' Vs ' + row["team2"]] = row["match_id"]
    match_ids_dict[row["team2"] + ' Vs ' + row["team1"]] = row["match_id"]


Create Batting DataFrame

In [36]:
# Load data and create DataFrame for BATTING_SUMMARY
with open('Data/t20_json_files/t20_wc_batting_summary.json') as f:
    data = json.load(f)

# Lets make the list flat
all_records = []
for record in data:
    all_records.extend(record['battingSummary'])

df_batting = pd.DataFrame(all_records)

# Lets replace dismissal column into out/not_out column 
df_batting["out/not_out"] = df_batting["dismissal"].apply(lambda x: "out" if len(x) >0 else "not_out")

# Lets drop dismissal column
df_batting.drop(columns=["dismissal"], inplace=True) 

# Lets remove some unwanted characters (â€)
df_batting["batsmanName"] = df_batting["batsmanName"].apply(lambda x: x.replace('â€',''))

df_batting.head(12)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out
5,Namibia Vs Sri Lanka,Namibia,6,Jan Frylinck,44,28,4,0,157.14,out
6,Namibia Vs Sri Lanka,Namibia,7,David Wiese,0,1,0,0,0.00,out
7,Namibia Vs Sri Lanka,Namibia,8,JJ Smit,31,16,2,2,193.75,not_out
8,Namibia Vs Sri Lanka,Sri Lanka,1,Pathum Nissanka,9,10,1,0,90.00,out
9,Namibia Vs Sri Lanka,Sri Lanka,2,Kusal Mendis,6,6,0,0,100.00,out


Now need to combine both table with each other. In both table thera are teams. So we need to link match with team 1 and team 2 

In [37]:
df_batting["match_id"] = df_batting["match"].map(match_ids_dict)
df_batting.head(4)

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50.00,out,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100.00,out,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out,T20I # 1823


Export final tables into /Data/Processed_data


In [38]:
df_batting.to_csv("Data/Processed_data/fact_batting_summary.csv", index=False)
df_match.to_csv("Data/Processed_data/fact_match_summary.csv", index=False)
